In [211]:
from bs4 import BeautifulSoup as bs
import requests

In [212]:
r =  requests.get('https://en.wikipedia.org/wiki/List_of_Walt_Disney_Pictures_films')
r

<Response [200]>

In [213]:
#convert to beautiful soup object
soup = bs(r.content)
soup

<!DOCTYPE html>

<html class="client-nojs" dir="ltr" lang="en">
<head>
<meta charset="utf-8"/>
<title>List of Walt Disney Pictures films - Wikipedia</title>
<script>document.documentElement.className="client-js";RLCONF={"wgBreakFrames":false,"wgSeparatorTransformTable":["",""],"wgDigitTransformTable":["",""],"wgDefaultDateFormat":"dmy","wgMonthNames":["","January","February","March","April","May","June","July","August","September","October","November","December"],"wgRequestId":"170b87ad-8e61-44d9-9b1c-fb1f35a4b9b4","wgCSPNonce":false,"wgCanonicalNamespace":"","wgCanonicalSpecialPageName":false,"wgNamespaceNumber":0,"wgPageName":"List_of_Walt_Disney_Pictures_films","wgTitle":"List of Walt Disney Pictures films","wgCurRevisionId":1130810861,"wgRevisionId":1130810861,"wgArticleId":1970335,"wgIsArticle":true,"wgIsRedirect":false,"wgAction":"view","wgUserName":null,"wgUserGroups":["*"],"wgCategories":["Webarchive template wayback links","CS1 maint: url-status","Articles with short descripti

In [214]:
contents = soup.prettify()
print(contents)

<!DOCTYPE html>
<html class="client-nojs" dir="ltr" lang="en">
 <head>
  <meta charset="utf-8"/>
  <title>
   List of Walt Disney Pictures films - Wikipedia
  </title>
  <script>
   document.documentElement.className="client-js";RLCONF={"wgBreakFrames":false,"wgSeparatorTransformTable":["",""],"wgDigitTransformTable":["",""],"wgDefaultDateFormat":"dmy","wgMonthNames":["","January","February","March","April","May","June","July","August","September","October","November","December"],"wgRequestId":"170b87ad-8e61-44d9-9b1c-fb1f35a4b9b4","wgCSPNonce":false,"wgCanonicalNamespace":"","wgCanonicalSpecialPageName":false,"wgNamespaceNumber":0,"wgPageName":"List_of_Walt_Disney_Pictures_films","wgTitle":"List of Walt Disney Pictures films","wgCurRevisionId":1130810861,"wgRevisionId":1130810861,"wgArticleId":1970335,"wgIsArticle":true,"wgIsRedirect":false,"wgAction":"view","wgUserName":null,"wgUserGroups":["*"],"wgCategories":["Webarchive template wayback links","CS1 maint: url-status","Articles wit

In [215]:
central_block = soup.find(id="mw-content-text")
print(central_block)

<div class="mw-body-content mw-content-ltr" dir="ltr" id="mw-content-text" lang="en"><div class="mw-parser-output"><p class="mw-empty-elt">
</p>
<style data-mw-deduplicate="TemplateStyles:r1033289096">.mw-parser-output .hatnote{font-style:italic}.mw-parser-output div.hatnote{padding-left:1.6em;margin-bottom:0.5em}.mw-parser-output .hatnote i{font-style:normal}.mw-parser-output .hatnote+link+.hatnote{margin-top:-0.5em}</style><div class="hatnote navigation-not-searchable" role="note">"List of Disney films" redirects here. For a broader list of Disney films, see <a href="/wiki/Lists_of_Walt_Disney_Studios_films" title="Lists of Walt Disney Studios films">Lists of Walt Disney Studios films</a>.</div>
<p class="mw-empty-elt">
</p>
<div class="floatright"><a class="image" href="/wiki/File:Walt_Disney_Pictures_text_logo.svg"><img alt="Walt Disney Pictures text logo.svg" data-file-height="101" data-file-width="300" decoding="async" height="74" src="//upload.wikimedia.org/wikipedia/commons/thu

                                                            Get movie title

In [216]:
#To get movie title
title =[]
def td_data(data):
    for i in data:
        # print(i.find('a').get_text(" ", strip=True))
        title.append(i.get_text(" ", strip=True))

In [217]:
info_table = central_block.find_all('tbody')

for index, table in enumerate(info_table):
    td_data(table.find_all('i'))

In [218]:
print(title)

['Snow White and the Seven Dwarfs', 'Pinocchio', 'Fantasia', 'The Reluctant Dragon', 'Dumbo', 'Bambi', 'Saludos Amigos', 'Victory Through Air Power', 'The Three Caballeros', 'Make Mine Music', 'Song of the South', 'Fun and Fancy Free', 'Melody Time', 'So Dear to My Heart', 'The Adventures of Ichabod and Mr. Toad', 'Cinderella', 'Treasure Island', 'Alice in Wonderland', 'The Story of Robin Hood', 'Peter Pan', 'The Sword and the Rose', 'The Living Desert', 'Rob Roy: The Highland Rogue', 'The Vanishing Prairie', '20,000 Leagues Under the Sea', 'Davy Crockett: King of the Wild Frontier', 'Lady and the Tramp', 'The African Lion', 'The Littlest Outlaw', 'The Great Locomotive Chase', 'Davy Crockett and the River Pirates', 'Secrets of Life', 'Westward Ho the Wagons!', 'Johnny Tremain', 'Perri', 'Old Yeller', 'The Light in the Forest', 'White Wilderness', 'Tonka', 'Sleeping Beauty', 'The Shaggy Dog', "Darby O'Gill and the Little People", 'Zorro the Avenger', 'Third Man on the Mountain', 'Toby T

                                                            Get Hyperlink

In [219]:
URL = []
def a_data(data):
    for i in data:
        url_initial = 'https://en.wikipedia.org/'
        URL.append(url_initial+i['href'])


In [220]:
info_table = central_block.find_all('tbody')

for index, table in enumerate(info_table):
    a_data(table.find_all('a',href=True))


In [221]:
for url in URL[:1]:
    print(url)


https://en.wikipedia.org//wiki/Snow_White_and_the_Seven_Dwarfs_(1937_film)


In [222]:
movies=[]
movie_info ={}
unfetched_url=[]

for url in URL[0:5]:
    print(url)
    r =  requests.get(f'{url}')
    soup = bs(r.content)
    contents = soup.prettify()
    info_box = soup.find(class_='infobox vevent')

    try:
        info_row = info_box.find_all("tr")

        def get_content_value(row_data):
            if row_data.find("li"):
                return [li.get_text(" ", strip=True).replace('\xa0',' ') for li in row_data.find_all('li')]
            else:
                return row_data.get_text(" ", strip=True).replace('\xa0',' ')

        for index,row in enumerate(info_row):
            if index==0:
                movie_info['Title'] = row.find("th").get_text(" ", strip=True)
            elif index==1:
                continue
            else:
                contenet_key = row.find("th").get_text(" ", strip=True)
                contenet_value = get_content_value(row.find("td"))
                movie_info[contenet_key] = contenet_value

        movies.append(movie_info)
        # print(movies)
    except AttributeError as e:
        print(f'Someting wrong with this URL -> {url}')
        print(e)
        unfetched_url.append(url)

https://en.wikipedia.org//wiki/Snow_White_and_the_Seven_Dwarfs_(1937_film)
https://en.wikipedia.org//wiki/RKO_Pictures
Someting wrong with this URL -> https://en.wikipedia.org//wiki/RKO_Pictures
'NoneType' object has no attribute 'find_all'
https://en.wikipedia.org//wiki/Pinocchio_(1940_film)
https://en.wikipedia.org//wiki/Fantasia_(1940_film)
https://en.wikipedia.org//wiki/The_Reluctant_Dragon_(1941_film)


In [223]:
movies

[{'Title': 'The Reluctant Dragon',
  'Directed by': 'Alfred Werker (live action) Hamilton Luske (animation) Jack Cutting , Ub Iwerks , Jack Kinney (sequence directors)',
  'Written by': 'Live-action: Ted Sears Al Perkins Larry Clemmons Bill Cottrell Harry Clork Robert Benchley The Reluctant Dragon segment: Kenneth Grahame (original book) Erdman Penner T. Hee Baby Weems segment: Joe Grant Dick Huemer John Miller',
  'Based on': 'The Adventures of Pinocchio by Carlo Collodi',
  'Produced by': 'Walt Disney',
  'Starring': 'Robert Benchley Frances Gifford Buddy Pepper Nana Bryant',
  'Music by': 'Frank Churchill Larry Morey',
  'Production company': 'Walt Disney Productions',
  'Distributed by': 'RKO Radio Pictures',
  'Release dates': ['February 7, 1940 ( 1940-02-07 ) ( Center Theatre ) [1]',
   'February 23, 1940 ( 1940-02-23 ) (United States) [2]'],
  'Running time': '74 minutes',
  'Country': 'United States',
  'Language': 'English',
  'Budget': '$600,000 [2]',
  'Box office': '$960,00